# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
# sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+ sep_local + '..') # For Windows import
os.chdir('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+ sep_local + '..') # For Linux import
print(sep_local)
print(os.getcwd())

env: TF_KERAS=1
/
/home/azeghost/git/azeghost/Generative_Models


In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.1


# Dataset loading

In [3]:
dataset_name='atari_pacman'

In [4]:
images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
#images_dir = 'C:\\projects\\pokemon\DS06\\'
validation_percentage = 25
valid_format = 'png'

In [8]:
from training.generators.from_images.file_image_generator import create_image_lists, get_generators

In [9]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose=0 
)

In [10]:
scale=1
image_size=(160//scale, 210//scale, 3)

batch_size = 10
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape = image_size
latents_dim = 30
intermediate_dim = 30

In [11]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode_flat', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

  INFO     | Found 1172401 training files

  INFO     | Found 381757 validation files



In [12]:
import tensorflow as tf

In [13]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [14]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [15]:
_instance_scale = 1.0

In [16]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [17]:
inputs_shape

(160, 210, 3)

# Model's Layers definition

In [18]:
# tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))

In [19]:
# enc_lays = [tdDense(units=intermediate_dim//2, activation='relu'),
#             tdDense(units=intermediate_dim//2, activation='relu'),
#             tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(units=latents_dim)]

# dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
#             tf.keras.layers.Reshape(inputs_shape),
#             tdDense(units=intermediate_dim, activation='relu'),
#             tdDense(units=_outputs_shape),
#             tf.keras.layers.Reshape(inputs_shape)
#             ]

In [20]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=3*intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [28]:
model_name = dataset_name+'AE_Dense'
#windows
#experiments_dir='..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+sep_local+'experiments'+sep_local + model_name

#linux 
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [29]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

In [30]:
# inputs_shape=image_size

In [31]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [32]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
# _restore = os.path.join(experiments_dir, 'var_save_dir')
_restore = os.path.join('/home/azeghost/git/Thesis_Plotting/experiments_FALL/atari_pacmanAE_Dense' , 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)

print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

Restore_dir /home/azeghost/git/Thesis_Plotting/experiments_FALL/atari_pacmanAE_Dense/var_save_dir
Recording_dir /home/azeghost/git/azeghost/Generative_Models/experiments/atari_pacmanAE_Dense
Current working dir /home/azeghost/git/azeghost/Generative_Models


In [33]:
#to restore trained model, set filepath=_restore

In [34]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size * EPIS_LEN,
    episode_len= 1, 
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...

  WARNING  | None
  WARNING  | None
Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 160, 210, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 160, 210, 15)      60        
_________________________________________________________________
dense_1 (Dense)              (None, 160, 210, 15)      240       
_________________________________________________________________
flatten (Flatten)            (None, 504000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                15120030  
_________________________________________________________________
batch_normalization (BatchNo (None, 30)                120       
__________________________________________________

TypeError: ('Keyword argument not understood:', 'inputs')

In [ ]:
#ae.compile(metrics=None)
ae.compile()

# Callbacks

In [ ]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

In [ ]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=int(1e4),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=int(1e4)
)

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)